In [1]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [2]:
# Sample dataset: Study hours, previous exam scores, and pass/fail labels
data = {
    'StudyHours': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'PrevExamScore': [30, 40, 45, 50, 60, 65, 70, 75, 80, 85],
    'Pass': [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]  # 0 = Fail, 1 = Pass
}

df = pd.DataFrame(data)

# Features and target variable
X = df[['StudyHours', 'PrevExamScore']]  # Features
y = df['Pass']  # Target variable

In [3]:
# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Note: The data here need feature standardization. Below is the he code without standardization. ###

In [4]:
# Initialize the LASSO model with alpha (regularization parameter)
lasso_model = Lasso(alpha=0.1)

# Train the model on the training data
lasso_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = lasso_model.predict(X_test)

# Evaluate the model's performance using R-squared
r2 = r2_score(y_test, y_pred)
print(f'R-squared score: {r2}')

# Display the coefficients of the features
print(f'LASSO Coefficients: {lasso_model.coef_}')

R-squared score: 0.9997884297520662
LASSO Coefficients: [0.         0.02463636]


In [5]:
# Try different alpha values and compare the results
for alpha in [0.01, 0.05, 0.1, 0.5, 1.0]:
    lasso_model = Lasso(alpha=alpha)
    lasso_model.fit(X_train, y_train)
    y_pred = lasso_model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    print(f'Alpha: {alpha}, R-squared score: {r2}, Coefficients: {lasso_model.coef_}')

Alpha: 0.01, R-squared score: 0.9981491660809606, Coefficients: [0.08153909 0.01180619]
Alpha: 0.05, R-squared score: 0.9999471074380165, Coefficients: [0.         0.02481818]
Alpha: 0.1, R-squared score: 0.9997884297520662, Coefficients: [0.         0.02463636]
Alpha: 0.5, R-squared score: 0.9947107438016529, Coefficients: [0.         0.02318182]
Alpha: 1.0, R-squared score: 0.9788429752066116, Coefficients: [0.         0.02136364]


### Pipeline + StandardScaler + LassoCV ###

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.metrics import r2_score
import numpy as np

# Sample dataset
data = {
    'StudyHours': [1,2,3,4,5,6,7,8,9,10],
    'PrevExamScore': [30,40,45,50,60,65,70,75,80,85],
    'Pass': [0,0,0,0,0,1,1,1,1,1]  # 0/1 target; LASSO treats this as regression
}
df = pd.DataFrame(data)

X = df[['StudyHours', 'PrevExamScore']]
y = df['Pass']

# Split once; do not use test set for tuning
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Pipeline: standardize inside CV + LassoCV to tune alpha fairly
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("lasso", LassoCV(alphas=[0.001, 0.01, 0.05, 0.1, 0.5, 1.0],
                      cv=5, max_iter=5000, random_state=42))
])

pipe.fit(X_train, y_train)

# Best alpha chosen via CV (no test leakage)
best_alpha = pipe.named_steps["lasso"].alpha_
coefs = pipe.named_steps["lasso"].coef_

print("Best alpha (CV):", best_alpha)
print("Standardized LASSO coefficients:", dict(zip(X.columns, coefs)))

# Evaluate on the held-out test set
y_pred = pipe.predict(X_test)
print("Test R^2:", r2_score(y_test, y_pred))

# (Optional) CV performance of the whole pipeline on training data
cv_scores = cross_val_score(pipe, X_train, y_train, cv=5, scoring="r2")
print("CV R^2 (mean±std):", f"{np.mean(cv_scores):.3f} ± {np.std(cv_scores):.3f}")


Best alpha (CV): 0.1
Standardized LASSO coefficients: {'StudyHours': np.float64(0.3178145108983667), 'PrevExamScore': np.float64(0.0)}
Test R^2: 0.969805176753795
CV R^2 (mean±std): nan ± nan


/Users/isdance/Desktop/projects/coursera-microsoft-ml-ai/.venv/lib/python3.12/site-packages/sklearn/metrics/_regression.py:1283: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/isdance/Desktop/projects/coursera-microsoft-ml-ai/.venv/lib/python3.12/site-packages/sklearn/metrics/_regression.py:1283: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
